In [ ]:
import awkward as ak
import cupy as cp
import numpy as np

from coffea.jitters import hist as gpu_hist

In [ ]:
N_dims = 4

test_gpu = ak.Array(cp.random.multivariate_normal(
    mean=np.zeros(shape=N_dims), 
    cov=np.eye(N_dims),
    size=(200_000_000//N_dims),
))

In [ ]:
hist_gpu = gpu_hist.Hist(
    "test", 
    gpu_hist.Bin("x", "x coordinate", 32, -5, 5),
    gpu_hist.Bin("y", "y coordinate", 32, -5, 5),
    gpu_hist.Bin("z", "z coordinate", 32, -5, 5),
    gpu_hist.Bin("t", "t coordinate", 32, -5, 5),
)

In [ ]:
%%timeit
hist_gpu.fill(x=test_gpu[:, 0], y=test_gpu[:,1], z=test_gpu[:,2], t=test_gpu[:,3])

In [ ]:
%%timeit
hist_gpu_cupy = cp.histogramdd(
    ak.to_cupy(test_gpu),
    bins=(32, 32, 32, 32),
    range=[(-5, 5), (-5, 5), (-5, 5), (-5, 5)]
)

In [ ]:
import hist

In [ ]:
test_cpu = ak.to_backend(test_gpu, "cpu")

In [ ]:
hist_cpu = hist.new.Reg(32, -5, 5).Reg(32, -5, 5).Reg(32, -5, 5).Reg(32, -5, 5).Weight()

In [ ]:
%%timeit
hist_cpu.fill(test_cpu[:,0], test_cpu[:,1], test_cpu[:,2], test_cpu[:,3])

In [ ]:
hist_gpu.values()[()].get()

In [ ]:
gpu_hist.plot1d(hist_gpu)

In [ ]:
import cudf
cudf.__version__

In [1]:
import uproot
import awkward as ak
#import cupy as cp
import cudf
import pyarrow.parquet as pq
import numpy as np
from coffea.jitters import hist

from ak_from_cudf import cudf_to_awkward

root_filepath = "/uscms_data/d2/lagray/Run2012B_SingleMu.root:Events"
#filepath = "/uscms_data/d3/fstrug/temp/Run2012B_SingleMu_compressed_zstd.parquet"
#filepath = "/uscms_data/d3/fstrug/temp/Run2012B_SingleMu_compressed_zstdlv3_PPv2-0_PLAIN.parquet"
#filepath = "/uscms_data/d3/fstrug/temp/Run2012B_SingleMu_compressed_zstdlv3_Pv2-6_PPv2-0_PLAIN.parquet"
filepath = "/uscms_data/d2/lagray/Run2012B_SingleMu_compressed_zstdlv3_PPv2-0_PLAIN.parquet"

In [ ]:
%%time
# Q1

#MET_pt = ak.to_backend(ak.from_arrow(pq.read_table(filepath, columns=["MET_pt"])["MET_pt"]), "cuda")
MET_pt = cudf_to_awkward(cudf.read_parquet(filepath, columns = ["MET_pt"])["MET_pt"])
q1_hist = hist.Hist(
    "Counts",
    hist.Bin("met", "$E_{T}^{miss}$ [GeV]", 100, 0, 200),
)
q1_hist.fill(met=MET_pt)

q1_hist.to_hist().plot1d(flow="none");

In [ ]:
%%time
# Q2

#Jet_pt = ak.to_backend(uproot.open("/uscms_data/d2/lagray/Run2012B_SingleMu.root:Events")["Jet_pt"].array(), "cuda")
#Jet_pt = ak.to_backend(ak.from_arrow(pq.read_table(filepath, columns=["Jet_pt"])["Jet_pt"]), "cuda")
Jet_pt = cudf_to_awkward(cudf.read_parquet(filepath, columns = ["Jet_pt"])["Jet_pt"])
q2_hist = hist.Hist(
    "Counts",
    hist.Bin("ptj", "Jet $p_{T}$ [GeV]", 100, 0, 200),
)
q2_hist.fill(ptj=ak.flatten(Jet_pt))

q2_hist.to_hist().plot1d(flow="none");

In [ ]:
%%time
# Q3

#jets = uproot.open(
#    "/uscms_data/d2/lagray/Run2012B_SingleMu.root:Events"
#).arrays(
#    ["Jet_pt","Jet_eta"],
#)
#Jet_pt = ak.to_backend(jets.Jet_pt, "cuda")
#Jet_eta = ak.to_backend(jets.Jet_eta, "cuda")

#table = ak.to_backend(ak.from_arrow(pq.read_table(filepath, columns=["Jet_pt", "Jet_eta"])), "cuda")
#Jet_pt = table.Jet_pt
#Jet_eta = table.Jet_eta

table = cudf.read_parquet(filepath, columns = ["Jet_pt", "Jet_eta"])
Jet_pt = cudf_to_awkward(table["Jet_pt"])
Jet_eta = cudf_to_awkward(table["Jet_eta"])

q3_hist = hist.Hist(
    "Counts",
    hist.Bin("ptj", "Jet $p_{T}$ [GeV]", 100, 0, 200),
)
q3_hist.fill(ptj=ak.flatten(Jet_pt[abs(Jet_eta) < 1.0]))

q3_hist.to_hist().plot1d(flow="none");

In [ ]:
%%time
# Q4

#jetmet = uproot.open(
#    "/uscms_data/d2/lagray/Run2012B_SingleMu.root:Events"
#).arrays(
#    ["Jet_pt","MET_pt"],
#)

#table = ak.to_backend(ak.from_arrow(pq.read_table(filepath, columns=["Jet_pt", "MET_pt"])), "cuda")
#Jet_pt = table.Jet_pt
#MET_pt = table.MET_pt

table = cudf.read_parquet(filepath, columns = ["Jet_pt", "MET_pt"])
Jet_pt = cudf_to_awkward(table["Jet_pt"])
MET_pt = cudf_to_awkward(table["MET_pt"])

q4_hist = hist.Hist(
    "Counts",
    hist.Bin("met", "$E_{T}^{miss}$ [GeV]", 100, 0, 200),
)
has2jets = ak.sum(Jet_pt > 40, axis=1) >= 2
q4_hist.fill(met=MET_pt[has2jets])

q4_hist.to_hist().plot1d(flow="none");

In [2]:
%%time
# Q5

from coffea.nanoevents.methods import candidate

table = cudf.read_parquet(
    filepath, 
    columns = [
        "MET_pt", 
        "Muon_pt", 
        "Muon_eta", 
        "Muon_phi", 
        "Muon_mass",
        "Muon_charge",
    ]
)
MET_pt = cudf_to_awkward(table["MET_pt"])
Muon_pt = cudf_to_awkward(table["Muon_pt"])
Muon_eta = cudf_to_awkward(table["Muon_eta"])
Muon_phi = cudf_to_awkward(table["Muon_phi"])
Muon_mass = cudf_to_awkward(table["Muon_mass"])
Muon_charge = cudf_to_awkward(table["Muon_charge"])

q5_hist = hist.Hist(
    "Counts",
    hist.Bin("met", "$E_{T}^{miss}$ [GeV]", 100, 0, 200),
)

Muon = ak.zip(
    {
        "pt": Muon_pt,
        "eta": Muon_eta,
        "phi": Muon_phi,
        "mass": Muon_mass,
        "charge": Muon_charge,
    },
    with_name="PtEtaPhiMCandidate",
    behavior=candidate.behavior,
)[0:10]

mupair = ak.combinations(Muon, 2, fields=["mu1", "mu2"])
pairmass = (mupair.mu1 + mupair.mu2).mass
goodevent = ak.any(
    (pairmass > 60)
    & (pairmass < 120)
    & (mupair.mu1.charge == -mupair.mu2.charge),
    axis=1,
)

q5_hist.fill(MET_pt[goodevent])


q5_hist.to_hist().plot1d(flow="none");

/home/lagray/coffea-gpu/coffea/src/coffea/nanoevents/methods/candidate.py:11: FutureWarning: In version 2024.7.0 (target date: 2024-06-30 11:59:59-05:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


ValueError: cannot convert to RegularArray because subarray lengths are not regular (in compiled code: https://github.com/scikit-hep/awkward/blob/awkward-cpp-35/awkward-cpp/src/cpu-kernels/awkward_ListOffsetArray_toRegularArray.cpp#L22)

In [ ]:
jetmet = uproot.open(
    "/uscms_data/d2/lagray/Run2012B_SingleMu.root:Events"
).arrays(
    ["Jet_pt","MET_pt"],
)

Jet_pt = ak.to_backend(jetmet.Jet_pt, "cuda")

ak.combinations(Jet_pt, 2)

In [ ]:
import fastparquet

In [ ]:
from hepconvert import root_to_parquet

root_to_parquet(in_file = "/uscms_data/d3/fstrug/temp/Run2012B_SingleMu.root",
                out_file = "/uscms_data/d2/lagray/Run2012B_SingleMu_compressed_zstdlv3_PPv2-0_PLAIN.parquet",
                tree="Events",
                compression = "zstd",
                compression_level = 3,
                extensionarray=False,
                parquet_version="2.6",
                parquet_page_version="2.0",
                parquet_extra_options = {"column_encoding": "PLAIN"}
               )

In [ ]:
import torch

In [1]:
import awkward as ak
import cupy as cp
import numba as nb

ak.numba.register_and_check()

@nb.vectorize(
    [
        nb.float32(nb.float32),
        nb.float64(nb.float64),
    ]
)
def _square(x):
    return x * x

@nb.vectorize(
    [
        nb.float32(nb.float32),
        nb.float64(nb.float64),
    ],
    target="cuda",
)
def _square_cuda(x):
    return x * x

def square_cuda_wrapped(x):
    counts = x.layout.offsets.data[1:] - x.layout.offsets.data[:-1]
    return ak.unflatten(cp.array(_square_cuda(ak.flatten(x))), counts)

counts = cp.random.poisson(lam=3, size=5000000)
flat_values = cp.random.normal(size=int(counts.sum()))

values = ak.unflatten(flat_values, counts)

values2_cpu = _square(ak.to_backend(values, "cpu"))

print(values2_cpu)

values2 = square_cuda_wrapped(values)

print(values2)

[[0.151], [0.00807, 2.34], ..., [6.53], [0.0751, 1.42, 0.0131, 1.78, 0.0162]]


LinkerError: [222] Call to cuLinkAddData results in CUDA_ERROR_UNSUPPORTED_PTX_VERSION
ptxas application ptx input, line 9; fatal   : Unsupported .version 8.4; current version is '8.2'

In [16]:
import awkward as ak
import cupy as cp
import numpy as np

counts = cp.random.poisson(lam=3, size=5000000)
flat_values = cp.random.normal(size=int(counts.sum()))

values = ak.unflatten(flat_values, counts)

np_vals = np.abs(values)
print(np_vals, ak.backend(np_vals))

cp_vals = cp.abs(values)
print(cp_vals, ak.backend(cp_vals))

[[0.045683051144471104, 0.27795220821182787, ..., 0.8520697217330707], ..., []] cuda


TypeError: Unsupported type <class 'awkward.highlevel.Array'>

In [ ]:
values

In [ ]:
dir(nb.cuda)

In [28]:
dir(values2)

['T',
 '__array__',
 '__class__',
 '__cuda_array_interface__',
 '__cuda_memory__',
 '__cuda_ndarray__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_default_stream',
 '_do_getitem',
 '_do_setitem',
 '_dummy',
 '_numba_type_',
 'alloc_size',
 'as_cuda_arg',
 'bind',
 'copy_to_device',
 'copy_to_host',
 'device_ctypes_pointer',
 'dtype',
 'flags',
 'get_ipc_handle',
 'getitem',
 'gpu_data',
 'is_c_contiguous',
 'is_f_contiguous',
 'nbytes',
 'ndim',
 'ravel',
 'reshape',
 'setitem',
 'shape',
 'size',
 'split',
 'squeeze',
 'stream',
 'strides',
 'transpose',
 'view']

In [1]:
import cupy as cp
import numpy as np

cp.float32 == np.float32

True